In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
from datetime import datetime

import backtrader as bt
import pandas as pd

from strategies import mystrategies

## Instantiate Cerebro

In [19]:
cerebro = bt.Cerebro()

## Configure Cerebro

In [20]:
cerebro.broker.set_cash(100)
cerebro.addsizer(bt.sizers.PercentSizer, percents=20)
cerebro.broker.setcommission(commission=0.005)

## Add data to Cerebro

In [4]:
class PandasData_Extend(bt.feeds.PandasData):
    lines = (
        'Open',
        'High',
        'Low',
        'Close',
        'Volume',
    )
    params = (
        ('open_time', None),
        ('Open', -1),
        ('High', -1),
        ('Low', -1),
        ('Close', -1),
        ('Volume', -1),
    )

In [5]:
def add_data_to_cerebro(sample, symbol:str, timeframe:str):
    path='/home/llagask/Trading/polaris_beta/datasets/'
    df = pd.read_pickle(f"{path}df_continuous_klines_{symbol}_{timeframe}.pckl")
    if isinstance(sample, dict):
        df = df.loc[sample.get('start') : sample.get('end')]
    elif isinstance(sample, int):
        df = df.iloc[-sample:]
    tframe = bt.TimeFrame.Days if (timeframe=='1d') else bt.TimeFrame.Minutes
    compression = 1 if (timeframe=='1d') else int(timeframe[:-1])
    data = PandasData_Extend(
        dataname = df,
        name = f"{symbol}_{timeframe}",
        timeframe = tframe, 
        compression = compression,
    )
    return data

In [21]:
symbol='BNBBUSD'
timeframe='15m'
sample={
    'start':'2022-8-1',
    'end':'2022-9-13'
}

data0 = add_data_to_cerebro(sample=sample, symbol=symbol, timeframe=timeframe)
cerebro.adddata(data0)

## Add analyzers to Cerebro

In [22]:
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
# cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# cerebro.addanalyzer(bt.analyzers.SQN, _name='systemquality')

## Strategy and Parameters range

In [23]:
cerebro.optstrategy(mystrategies.AroonPlusMa, 
    
    enter_long=True,
    enter_short=True,
    
    ema=range(100,300,15),
    
    aroon_timeperiod=range(50,200,10),
    
    leverage_factor = 1.0
)

In [ ]:
''' cerebro.optstrategy(mystrategies.EmaCrossTriple, 
    
    enter_long=False,
    enter_short=True,
    
    ema_slow=[100,120],
    ema_mid=[60,70,80],
    ema_fast=[25,35,40],
    
    tp_k=range(1,3),
    sl_k=range(1,3),
    
    # leverage_factor = 1.0
) '''

## Run Cerebro

In [24]:
optimizations = cerebro.run()

len(optimizations)

210

# Analyze Results

In [ ]:
# optimizations[0][0].analyzers.getnames()

In [25]:
all_params = [
    dict(zip(
        list(x[0].params._getkeys()),
        list(x[0].params._getvalues()),
    ))for x in optimizations 
]

In [26]:
len(all_params)

210

In [27]:
df_params = pd.DataFrame(all_params)
df_params.drop(columns=['verbose'], inplace=True)
df_params.head()

,futures_like,enter_long,enter_short,ema,aroon_timeperiod,leverage_factor,margin,tp_k,sl_k,trail
0,True,True,True,100,50,1.0,0.6,2,2,None
1,True,True,True,100,60,1.0,0.6,2,2,None
2,True,True,True,100,70,1.0,0.6,2,2,None
3,True,True,True,100,80,1.0,0.6,2,2,None
4,True,True,True,100,90,1.0,0.6,2,2,None


In [28]:
trades = [
    dict(
        pnl_net       = x[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total'], 
        total_trades  = x[0].analyzers.tradeanalyzer.get_analysis()['total']['total'], 
        won           = x[0].analyzers.tradeanalyzer.get_analysis()['won']['total'], 
        lost          = x[0].analyzers.tradeanalyzer.get_analysis()['lost']['total'],
        
        long_won      = x[0].analyzers.tradeanalyzer.get_analysis()['long']['won'],
        short_won     = x[0].analyzers.tradeanalyzer.get_analysis()['short']['won'],
        
        long_lost     = x[0].analyzers.tradeanalyzer.get_analysis()['long']['lost'],
        short_lost    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['lost'],
        
        longs_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['long']['pnl']['total'],
        shorts_pnl    = x[0].analyzers.tradeanalyzer.get_analysis()['short']['pnl']['total'],
        
        moneydown_max = x[0].analyzers.drawdown.get_analysis()['max']['moneydown'],
    )for x in optimizations
]

In [29]:
df_trades = pd.DataFrame(trades)

In [30]:
df_results = pd.concat([df_params, df_trades], axis=1)

In [31]:
df_results.columns

Index(['futures_like', 'enter_long', 'enter_short', 'ema', 'aroon_timeperiod',
       'leverage_factor', 'margin', 'tp_k', 'sl_k', 'trail', 'pnl_net',
       'total_trades', 'won', 'lost', 'long_won', 'short_won', 'long_lost',
       'short_lost', 'longs_pnl', 'shorts_pnl', 'moneydown_max'],
      dtype='object')

# BEST RESULTS

In [33]:
best = df_results[ 
    [
    'pnl_net',
    'won',
    'lost',
    'total_trades',
    'moneydown_max',
    'long_won',
    'long_lost',
    'short_won',
    'short_lost',
    'longs_pnl',
    'shorts_pnl',
    'leverage_factor',
    'aroon_timeperiod',
    'ema',
    ]
    ].nlargest(20,['pnl_net'])
    
best

,pnl_net,won,lost,total_trades,moneydown_max,long_won,long_lost,short_won,short_lost,longs_pnl,shorts_pnl,leverage_factor,aroon_timeperiod,ema
22,-2.258368,4,13,18,5.104410,1,3,3,10,0.499669,-2.758036,1.0,120,115
37,-2.258368,4,13,18,5.104410,1,3,3,10,0.499669,-2.758036,1.0,120,130
5,-2.379747,5,12,17,5.830073,2,4,3,8,0.047241,-2.426989,1.0,100,100
20,-2.379747,5,12,17,5.830073,2,4,3,8,0.047241,-2.426989,1.0,100,115
35,-2.379747,5,12,17,5.830073,2,4,3,8,0.047241,-2.426989,1.0,100,130
50,-2.379747,5,12,17,5.830073,2,4,3,8,0.047241,-2.426989,1.0,100,145
6,-2.639553,6,12,18,5.629137,2,3,4,9,-0.201402,-2.438151,1.0,110,100
21,-2.639553,6,12,18,5.629137,2,3,4,9,-0.201402,-2.438151,1.0,110,115
65,-2.778374,5,13,18,6.223312,2,4,3,9,0.016368,-2.794742,1.0,100,160
7,-2.796767,4,14,19,5.404950,1,4,3,10,-0.053923,-2.742844,1.0,120,100


In [ ]:
dict(df_results.loc[ int(best.first_valid_index()) ])

In [ ]:
df_results[
    [
    'pnl_net',
    'won',
    'lost',
    'total_trades',
    'moneydown_max',
    'long_won',
    'long_lost',
    'short_won',
    'short_lost',
    'longs_pnl',
    'shorts_pnl',
    'leverage_factor',
    'aroon_timeperiod',
    'ema',
    ]
].nsmallest(10,'pnl_net')

### <center>Failed test objects

In [ ]:
def identify_failed_objects(optimizations):
    failed_idx = []
    for idx,op in enumerate(optimizations):
        try:
            op[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total']
        except:
            failed_idx.append(idx)
    print('Failed tests #',len(failed_idx))
    return failed_idx

def pop_failed_objects(optimizations,failed_idx):
    failed_objects = []
    for f in failed_idx:
        failed_objects.append(optimizations.pop(f))
    print('#',len(failed))
    return failed_objects

In [ ]:
# len(optimizations)

In [ ]:
''' while True:
    indices = identify_failed_objects(optimizations)
    if not indices:
        break
    objects = pop_failed_objects(optimizations,failed_idx=indices)
    if not objects:
        break

print(len(objects)) '''

In [ ]:
''' dict(zip(
        list(objects[0][0].params._getkeys()),
        list(objects[0][0].params._getvalues()),
    )) '''